# Laboratorio 6 Parte 1

### Reducción de dimensión: Selección de características

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

## Estudiantes

#### Primer Integrante: Danny Francisco Hernandez Godoy

#### Segundo Integrante: Daniel Martinez

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de selección de características y usar SVM para resolver un problema de clasificación multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography


Analice la base de datos, sus características, su variable de salida y el contexto del problema.

Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: sudo pip install mlxtend. También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [3]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time

#cargamos la bd de entrenamiento
db = np.loadtxt('/content/DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

print('Dimensiones de la base de datos de entrenamiento. dim de X: ' + str(np.shape(X)) + '\tdim de Y: ' + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [4]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print(('\n\nTiempo total de ejecución: ' + str(time.time()-tiempo_i)) + ' segundos.')


Error de validación sin aplicar SFS: 0.07712817787226504 +/- 0.05442325724156325


Tiempo total de ejecución: 0.5736293792724609 segundos.


## Ejercicio 1

1.1 Describa la metodología de validación que se está aplicando.

R/: Validación cruzada Kfolds 10


    
1.2 Con qué modelo se está resolviendo el problema planteado? Cuáles son los parámetros establecidos para el modelo?

R/: SVC, vectores de soporte para la clasificacion, ya que se usa el OVR que es una contra el resto, el kernel es "rbf", C=100 el parametro que importancia o penalización, gamma=0.0001 es el coeficiente del kernel.

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de selección de características. https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#sequential-feature-selector

In [0]:
#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
def select_features(modelo, n_features, fwd, fltg):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0)
    
    return sfs

def invento (model, n_features, forward, floating):
  #Para calcular el costo computacional
  tiempo_i = time.time()

  #Implemetamos la metodología de validación 

  Errores = np.ones(10)
  j = 0
  kf = KFold(n_splits=10)

  for train_index, test_index in kf.split(X):

      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = Y[train_index], Y[test_index]  

      #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

      #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
      #Ejercicio 3.1
      sf = select_features(model,n_features, forward, floating) #Complete el código aquí
      
      #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
      #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
      sf = sf.fit(X_train,y_train)#Complete el código aquí
      nombre=sf.k_feature_idx_
      Errores[j] = 1-sf.k_score_
      j+=1

  print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

  print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )
  tiempor=time.time()-tiempo_i
  print ("\n\nTiempo total de ejecución: " + str(tiempor) + " segundos.")

  #print str(ypred)
  #print str(y_test)
  return np.mean(Errores), np.std(Errores), tiempor, nombre

In [5]:
ex=invento(clf, 7, True, True)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    3.1s finished
Features: 1/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 2/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s finished
Features: 3/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend w


Error de validación aplicando SFS: 0.019911828052138435 +/- 0.002727682121130486

Eficiencia en validación aplicando SFS: 97.91013584117032%


Tiempo total de ejecución: 130.0550389289856 segundos.


SFS = forward=True, floating=False
SBS = forward=False, floating=False
SFFS = forward=True, floating=True
SBFS = forward=False, floating=True

## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de características a seleccionar (use 3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS), para que complete la siguiente tabla de resultados:


In [7]:
import pandas as pd
#import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SBS','SVM + SBS','SVM + SBS','SVM + SFFS','SVM + SFFS','SVM + SFFS','SVM + SBFS','SVM + SBFS','SVM + SBFS']),
    '# de características seleccionadas' : pd.Series([22,3,7,10,3,7,10,3,7,10,3,7,10]),
    'forward' : pd.Series([None,True,True,True,False,False,False,True,True,True,True,True,True]),
    'floating' : pd.Series([None,False,False,False,False,False,False,True,True,True,True,True,True])
    })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de características seleccionadas', 'forward', 'floating'], inplace=True)
#df_types["Error de validación"][8] = "0.019"
#df_types["IC(std)"][8] = "0.002"
#df_types["Tiempo de ejecución"][8] = "107.9 s"

for i in df_types.index:
    print(i[0])
    print(i[1])
    print(i)
    #print(i[3])
    ED=invento(clf, i[1], i[2], i[3])
    df_types["Error de validación"][i] = float(ED[0])
    df_types["IC(std)"][i] = float(ED[1])
    df_types["Tiempo de ejecución"][i] = float(ED[2])

#df_types.sort_index(inplace=True)
#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


SVM sin selección
22
('SVM sin selección', 22, nan, nan)


[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    3.1s finished
Features: 1/22[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 2/22[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s finished
Features: 3/22[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:


Error de validación aplicando SFS: 0.007003175217029612 +/- 0.0008487540566555215

Eficiencia en validación aplicando SFS: 99.32079414838036%


Tiempo total de ejecución: 376.1552197933197 segundos.
SVM + SFS
3
('SVM + SFS', 3, True, False)


[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    3.0s finished
Features: 1/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.6s finished
Features: 2/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.3s finished
Features: 3/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    3.0s finished
Features: 1/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.5s finished
Features: 2/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.6s finished
Features: 3/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs

...


Error de validación aplicando SFS: 0.04311674890112802 +/- 0.005501557850665672

Eficiencia en validación aplicando SFS: 95.61128526645768%


Tiempo total de ejecución: 66.84933853149414 segundos.
SVM + SBFS
7
('SVM + SBFS', 7, True, True)


[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    3.1s finished
Features: 1/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 2/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s finished
Features: 3/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   


Error de validación aplicando SFS: 0.019911828052138435 +/- 0.002727682121130486

Eficiencia en validación aplicando SFS: 97.91013584117032%


Tiempo total de ejecución: 129.74345421791077 segundos.
SVM + SBFS
10
('SVM + SBFS', 10, True, True)


[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    3.0s finished
Features: 1/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 2/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s finished
Features: 3/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:


Error de validación aplicando SFS: 0.012804132315821847 +/- 0.0022432590153498955

Eficiencia en validación aplicando SFS: 98.58934169278997%


Tiempo total de ejecución: 175.64489221572876 segundos.


In [8]:
df_types

Error de validación  ... Tiempo de ejecución
Tecnica           # de características seleccionadas forward floating                      ...                    
SVM sin selección 22                                 NaN     NaN               0.00700318  ...             376.155
SVM + SFS         3                                  True    False              0.0431167  ...             62.9122
                  7                                  True    False              0.0199118  ...             109.192
                  10                                 True    False              0.0130655  ...             136.376
SVM + SBS         3                                  False   False              0.0442142  ...             178.126
                  7                                  False   False              0.0199121  ...             163.224
                  10                                 False   False              0.0112886  ...             145.641
SVM + SFFS        3                                  True    True               0.0431167  ...             66.7918
                  7                                  True    True               0.0199118  ...               129.5
                  10                                 True    True               0.0128041  ...             175.782
SVM + SBFS        3                                  True    True               0.0431167  ...             66.8493
                  7                                  True    True               0.0199118  ...             129.743
                  10                                 True    True               0.0128041  ...             175.645

[13 rows x 3 columns]

3.2 Según la teoría vista en el curso, se está usando una función tipo filtro o tipo wrapper y cuál es?

R/: Wrapper ya que la función objetivo es un modelo de aprendizaje y es una maquina de vectores de soporte.

3.3 Con los resultados de la tabla anterior haga un análisis de cuál es el mejor resultado teniendo en cuenta tanto la eficiencia en la clasificación como el costo computacional del modelo y la estrategia implementada.

R/:El mejor es el que usa la técnica SVM + SBS con 10 carácteristicas seleccionadas con un error de validacion de 0.011 con un intervalo de confianza de 0.001.

3.4 Haga uso del atributo sf.k_feature\_idx\_ (deje evidencia del código usado para esto) para identificar cuáles fueron las características seleccionadas en el mejor de los resultados encontrados. No presente los indices de las características sino sus nombres y descripción.

R/: 0 LB - FHR baseline (beats per minute) 1 AC - # of accelerations per second 2 FM - # of fetal movements per second 4 DL - # of light decelerations per second 7 ASTV - percentage of time with abnormal short term variability 9 ALTV - percentage of time with abnormal long term variability 12 Min - minimum of FHR histogram 18 Median - histogram median 19 Variance - histogram variance 21 CLASS - FHR pattern class code (1 to 10)

In [6]:
ex=invento(clf, 10, False, False)
print(ex[3])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    1.6s finished
Features: 21/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.5s finished
Features: 20/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.4s finished
Features: 19/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.3s finished
Features: 18/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.2s finished
Features: 17/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.1s finished
Features: 16/10[Par


Error de validación aplicando SFS: 0.011288625753178627 +/- 0.001282860778208819

Eficiencia en validación aplicando SFS: 98.90282131661442%


Tiempo total de ejecución: 146.6911346912384 segundos.
(0, 1, 2, 4, 7, 9, 12, 18, 19, 21)


3.5 De acuerdo a los resultados encontrados y la respuesta anterior, usted como ingeniero de datos que le puede sugerir a un médico que esté trabajando en un caso enmarcado dentro del contexto de la base de datos trabajada, para que apoye su diagnóstico?

R/: Lo mejor es quedarse con el modelo que tiene 10 caracteristicas, ya que aunque el tiempo de este es un poco mas grande que otros, este tiene menor error, asi que es mejor eliminar las otras caracteristicas.